In [28]:
!pip install kmodes
!pip install gower


In [11]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
adult = fetch_ucirepo(id=2) 
  
# data (as pandas dataframes) 
X = adult.data.features 
y = adult.data.targets 
  
df = X


In [57]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, MaxAbsScaler

df: pd.DataFrame = X

print(df.shape)
df = df[~df.eq("?").any(axis=1)]
df = df.dropna().reset_index(drop=True)

print(df.shape)

df.to_csv("adults.csv", index=False)

# Identify categorical and numerical columns
categorical_cols = df.select_dtypes(include=['object']).columns
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns


# df = pd.get_dummies(df, columns=categorical_cols)

# cat_cols = df.select_dtypes(include=['object', 'category']).columns
categorical_indexes = [df.columns.get_loc(col) for col in categorical_cols]

categorical_indexes

for col in categorical_cols:
    df[col], _ = pd.factorize(df[col])
    
# df[numerical_cols]  = MinMaxScaler().fit_transform(df[numerical_cols])
    
df

(48842, 14)
(45222, 14)


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,39,0,77516,0,13,0,0,0,0,0,2174,0,40,0
1,50,1,83311,0,13,1,1,1,0,0,0,0,13,0
2,38,2,215646,1,9,2,2,0,0,0,0,0,40,0
3,53,2,234721,2,7,1,2,1,1,0,0,0,40,0
4,28,2,338409,0,13,1,3,2,1,1,0,0,40,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45217,33,2,245211,0,13,0,3,3,0,0,0,0,40,0
45218,39,2,215419,0,13,2,3,0,0,1,0,0,36,0
45219,38,2,374983,0,13,1,3,1,0,0,0,0,50,0
45220,44,2,83891,0,13,2,0,3,2,0,5455,0,40,0


In [68]:
import gower
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score

new_df: pd.DataFrame = X

new_df = new_df[~new_df.eq("?").any(axis=1)]
new_df = new_df.dropna().reset_index(drop=True)
mini_df = new_df.sample(5000)

gower_dist = gower.gower_matrix(mini_df)


# Run DBSCAN on the Gower distance matrix
dbscan = DBSCAN(metric='precomputed', eps=0.1, min_samples=5)
labels = dbscan.fit_predict(gower_dist)



# score = silhouette_score(mini_df, labels)

print(labels)

set(labels)

for i in set(labels):
    count = [x for x in labels if x == i]
    print(f"{i}: {len(count)}")



[0 0 0 ... 0 0 0]
0: 4303
1: 5
2: 12
3: 4
4: 4
-1: 672


In [71]:
mask = labels != -1

# Compute silhouette score only on non-noise points
score = silhouette_score(gower_dist[mask][:, mask], labels[mask], metric='precomputed')
# score = silhouette_score(gower_dist, labels, metric='precomputed')

print("Silhouette Score:", score)

Silhouette Score: -0.016717758


In [74]:
from kmodes.kprototypes import KPrototypes
from sklearn.metrics import silhouette_score
import numpy as np

# Define the range of potential clusters and gamma values
clusters_range = range(2,15)
gamma_range = np.linspace(0.1, 1, 10) # gamma values between 0.1 and 1

# Placeholder variables
best_score = -1
best_clusters = None
best_labels = None
best_gamma = None

all_scores = []

mini_df = df.sample(10000)

for n_clusters in clusters_range:
    # for gamma in gamma_range:
    kproto = KPrototypes(n_clusters=n_clusters, init='Huang')
    clusters = kproto.fit_predict(mini_df, categorical=categorical_indexes)
    score = silhouette_score(mini_df, clusters)
    all_scores.append([n_clusters, score, kproto.cost_])

    # Check if this configuration beats the best score
    print(f"{score=}, {n_clusters=}, {kproto.cost_=}")
    if score > best_score:
        best_labels = clusters
        best_score = score
        best_clusters = n_clusters
        # best_gamma = gamma

print(f"Best score: {best_score}")
print(f"Optimal number of clusters: {best_clusters}")
print(f"Optimal gamma value: {best_gamma}")
print(f"{all_scores}")

score=0.5879314487445938, n_clusters=2, kproto.cost_=46519926248627.34
score=0.549697552444435, n_clusters=3, kproto.cost_=25313973469048.527
score=0.5552031118482339, n_clusters=4, kproto.cost_=16507028427320.635
score=0.5377135114449826, n_clusters=5, kproto.cost_=11741704274283.936
score=0.5424114692679833, n_clusters=6, kproto.cost_=8474958772991.547
score=0.5452824406453705, n_clusters=7, kproto.cost_=6388010631914.506
score=0.5267019767964408, n_clusters=8, kproto.cost_=5250316111419.136
score=0.5239333462120755, n_clusters=9, kproto.cost_=4366122597697.0127
score=0.5254722812749802, n_clusters=10, kproto.cost_=3493980705628.491
score=0.5154009558619855, n_clusters=11, kproto.cost_=3069157689652.015
score=0.5252923650141789, n_clusters=12, kproto.cost_=2683590205681.794
score=0.5215676975133386, n_clusters=13, kproto.cost_=2376471887535.2354
score=0.5241651349443052, n_clusters=14, kproto.cost_=2048019369316.4358
Best score: 0.5879314487445938
Optimal number of clusters: 2
Optima

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns

tsne = TSNE(n_components=2, random_state=53, perplexity=50)
tsne_on_df = tsne.fit_transform(mini_df)

def plot_tsne(X_tsne, title):
    plt.figure(figsize=(8,6))
    sns.scatterplot(x=X_tsne[:,0], y=X_tsne[:,1], hue=y_sample, palette='viridis', s=50)
    plt.title(title)
    plt.xlabel('t-SNE Component 1')
    plt.ylabel('t-SNE Component 2')
    plt.legend(title='Income')
    plt.show()

# Visualizations
plot_tsne(X_tsne_pca, 't-SNE on PCA-Reduced Data')